<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=95453bb49ae88e23bf3b09d5cda9ec86e46fb7b9492840ef5e26c6c3525295cd
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-16 14:17:04
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -25.48 K (-0.18%)
Current PnL: -29.05 L (-18.53%)
CY Booked + Current PnL: -14.76 L (-9.42%)
-------------------
Total profit:  1.88 L
Total loss:  -30.93 L
-------------------
Total Booked + Current PnL: 12.22 L (9.41%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.21%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.51 L (67.5%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.42,15.72,4.91,21.40,15081.0,41722.0,307148.0,7.34,62.0,X-LC,6.59,14.0,2.77,2.19,23.04,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.75,5.48,13.03,19.23,20680.0,8252.0,158712.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.93,XY25,ATH,MINING
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.82,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,54.0,M-SC,9.09,240.0,-1.00,1.07,30.51,OX40N,NTT,PAINTS
49,MASFIN,398.61,0.21,-4.53,27.84,22.05,26042.0,-4440.0,93540.0,-17.80,46.0,H-SC,5.54,168.0,-0.17,0.67,35.97,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,8.40,-1.50,109.87,106.72,133478.0,-1849.0,121487.0,-51.20,59.0,H-SC,8.08,138.0,-0.01,0.87,21.53,AR,ATH,MISC
41,INFY,1972.00,5.18,14.55,17.19,34.24,61612.0,45527.0,358415.0,-10.62,64.0,X-LC,7.69,6.0,0.74,2.56,24.04,X40,NTT,IT
55,RAJESHEXPO,518.00,5.00,-66.52,198.87,0.07,91341.0,-91247.0,45930.0,1583.08,42.0,L-SC,0.75,269.0,-1.00,0.33,14.23,OX40N,NTT,JEWELLERY
13,BSOFT,831.70,3.38,-21.35,91.72,50.79,101460.0,-30034.0,110619.0,0.89,54.0,H-SC,10.18,171.0,-0.30,0.79,28.80,XR,ATH,IT
84,WIPRO,420.00,2.81,9.72,57.01,72.27,105988.0,16464.0,185912.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.49,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1943.92,-3.93,-47.52,318.59,119.69,452736.0,-128655.0,142106.0,-47.02,24.0,H-SC,12.59,150.0,-0.28,1.02,12.26,AR,ATH,IT
22,DIXON,18931.72,-3.18,-25.47,76.11,31.26,130909.0,-58768.0,172000.0,-65.23,22.0,X-MC,21.55,56.0,-0.45,1.23,0.00,X40N,ATH,IT
45,JIOFIN,387.00,-2.49,-9.98,38.29,24.48,92153.0,-26694.0,240671.0,-11.49,34.0,X-LC,1.71,37.0,-0.29,1.72,39.50,XY24,BTT,FINANCE
4,ALKYLAMINE,4546.37,-2.33,-31.67,183.37,93.63,126505.0,-31974.0,68989.0,-26.89,49.0,H-SC,16.53,143.0,-0.25,0.49,4.30,SR,ATH,CHEMICALS
79,VAIBHAVGBL,521.00,-2.30,-33.26,140.90,60.77,171864.0,-60799.0,121976.0,40.50,32.0,H-SC,3.13,158.0,-0.35,0.87,9.85,XR,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.53,0.66,19.94,20.74,40781.0,1348.0,204518.0,33.49,57.0,M-MC,5.18,189.0,0.03,1.46,11.35,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.43,-1.18,110.39,107.91,161067.0,-1737.0,145907.0,-19.74,55.0,M-SC,9.95,220.0,-0.01,1.04,8.79,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.82,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,54.0,M-SC,9.09,240.0,-1.00,1.07,30.51,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,2.56,-18.63,62.26,32.03,66789.0,-24561.0,107274.0,-12.01,58.0,H-SC,6.55,157.0,-0.37,0.77,20.16,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.01,-22.27,28.76,0.09,22530.0,-22439.0,78338.0,81.86,37.0,M-SC,3.86,253.0,-1.00,0.56,2.92,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.07,-22.69,46.77,13.47,97508.0,-61182.0,208485.0,-68.31,47.0,H-SC,3.96,159.0,-0.63,1.49,6.01,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.53,0.66,19.94,20.74,40781.0,1348.0,204518.0,33.49,57.0,M-MC,5.18,189.0,0.03,1.46,11.35,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.75,5.48,13.03,19.23,20680.0,8252.0,158712.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.93,XY25,ATH,MINING
84,WIPRO,420.00,2.81,9.72,57.01,72.27,105988.0,16464.0,185912.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.49,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,2.81,9.72,57.01,72.27,105988.0,16464.0,185912.0,-5.63,56.0,M-LC,6.35,101.0,0.16,1.33,16.49,XR,NTT,IT
38,INDIAMART,4810.62,8.40,-1.50,109.87,106.72,133478.0,-1849.0,121487.0,-51.20,59.0,H-SC,8.08,138.0,-0.01,0.87,21.53,AR,ATH,MISC
25,FINCABLES,1641.55,0.43,-1.18,110.39,107.91,161067.0,-1737.0,145907.0,-19.74,55.0,M-SC,9.95,220.0,-0.01,1.04,8.79,OX40N,ATH,CABLES
37,IEX,219.00,-0.47,-5.86,57.87,48.63,108936.0,-11711.0,188243.0,-37.43,48.0,H-SC,13.25,137.0,-0.11,1.34,5.08,XR,NTT,MISC
49,MASFIN,398.61,0.21,-4.53,27.84,22.05,26042.0,-4440.0,93540.0,-17.80,46.0,H-SC,5.54,168.0,-0.17,0.67,35.97,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.39,-19.10,36.93,10.78,71803.0,-45889.0,194429.0,-51.78,12.0,X-LC,9.34,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
7,AWL,485.00,-1.98,-30.77,127.07,57.19,286880.0,-100362.0,225765.0,-67.33,18.0,X-MC,11.01,58.0,-0.35,1.61,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-3.18,-25.47,76.11,31.26,130909.0,-58768.0,172000.0,-65.23,22.0,X-MC,21.55,56.0,-0.45,1.23,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,-0.17,-41.68,131.82,35.20,98923.0,-53626.0,75044.0,-1.66,25.0,X-SC,17.60,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,-2.05,-12.22,32.64,16.43,63543.0,-27097.0,194677.0,-25.59,28.0,X-MC,7.45,66.0,-0.43,1.39,14.89,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JIOFIN,387.00,-2.49,-9.98,38.29,24.48,92153.0,-26694.0,240671.0,-11.49,34.0,X-LC,1.71,37.0,-0.29,1.72,39.50,XY24,BTT,FINANCE
73,TCS,4389.97,0.35,-11.91,37.02,20.71,123349.0,-45042.0,333195.0,-24.27,47.0,X-LC,1.88,3.0,-0.37,2.38,11.33,X40,ATH,IT
78,UNITDSPR,1644.00,0.43,-1.44,22.53,20.77,51993.0,-3372.0,230772.0,-10.02,37.0,X-MC,1.98,70.0,-0.06,1.65,5.38,X40N,NTT,BREWERIES
20,DABUR,735.00,0.00,0.44,43.07,43.70,107317.0,1101.0,249169.0,-7.78,54.0,X-MC,2.13,73.0,0.01,1.78,15.56,XY24,BTT,FMCG
65,SIEMENS,4671.50,-2.07,-21.83,60.57,25.51,88111.0,-40625.0,145470.0,-10.68,33.0,H-LC,2.51,50.0,-0.46,1.04,7.20,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.98,-30.77,127.07,57.19,286880.0,-100362.0,225765.0,-67.33,18.0,X-MC,11.01,58.0,-0.35,1.61,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.17,-41.68,131.82,35.20,98923.0,-53626.0,75044.0,-1.66,25.0,X-SC,17.60,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
9,BAJAJHLDNG,14451.52,-0.45,-4.39,35.15,29.22,63896.0,-8344.0,181781.0,-6.67,35.0,X-LC,11.98,29.0,-0.13,1.30,0.0,X40,ATH,FINANCE
8,BAJAJHFL,181.50,-0.76,-20.52,95.62,55.47,184885.0,-49932.0,193354.0,-29.54,32.0,X-MC,15.90,63.0,-0.27,1.38,0.0,X40N,ATH,FINANCE
22,DIXON,18931.72,-3.18,-25.47,76.11,31.26,130909.0,-58768.0,172000.0,-65.23,22.0,X-MC,21.55,56.0,-0.45,1.23,0.0,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.81,-30.07,104.27,42.84,47399.0,-19548.0,45458.0,-54.27,43.0,X-SC,24.88,83.0,-0.41,0.32,2.84,XY24,NTT,MISC
58,RELAXO,1176.00,-0.75,-49.47,201.42,52.31,147738.0,-71812.0,73348.0,-46.18,33.0,X-SC,8.04,92.0,-0.49,0.52,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.17,-41.68,131.82,35.20,98923.0,-53626.0,75044.0,-1.66,25.0,X-SC,17.60,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.04,-3.94,29.16,24.08,35561.0,-4997.0,121953.0,-11.75,42.0,X-SC,5.87,91.0,-0.14,0.87,14.91,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.15,-17.48,73.68,43.32,99026.0,-28472.0,134400.0,-29.05,47.0,X-SC,8.18,90.0,-0.29,0.96,3.19,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-1.39,-19.10,36.93,10.78,71803.0,-45889.0,194429.0,-51.78,12.0,X-LC,9.34,1.0,-0.64,1.39,0.00,X40,NTT,FMCG
75,TMPV,600.00,1.34,-27.34,69.25,22.98,113663.0,-61764.0,164134.0,-24.30,44.0,X-LC,3.64,2.0,-0.54,1.17,3.23,XY24,NTT,AUTO
73,TCS,4389.97,0.35,-11.91,37.02,20.71,123349.0,-45042.0,333195.0,-24.27,47.0,X-LC,1.88,3.0,-0.37,2.38,11.33,X40,ATH,IT
81,VBL,671.64,-0.01,1.47,33.70,35.67,108008.0,4657.0,320499.0,-10.61,61.0,X-LC,2.79,5.0,0.04,2.29,15.53,X40N,ATH,FMCG
41,INFY,1972.00,5.18,14.55,17.19,34.24,61612.0,45527.0,358415.0,-10.62,64.0,X-LC,7.69,6.0,0.74,2.56,24.04,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.72,-29.77,89.21,32.89,48373.0,-22982.0,54224.0,-769.87,74.0,L-MC,9.20,259.0,-0.48,0.39,49.33,XR,NTT,BANKS
13,BSOFT,831.70,3.38,-21.35,91.72,50.79,101460.0,-30034.0,110619.0,0.89,54.0,H-SC,10.18,171.0,-0.30,0.79,28.80,XR,ATH,IT
69,SURYODAY,216.00,0.90,-19.25,52.37,23.03,75724.0,-34476.0,144595.0,57.89,52.0,H-SC,4.44,165.0,-0.46,1.03,43.47,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.82,-14.05,16.36,0.02,24547.0,-24519.0,150040.0,128.79,54.0,M-SC,9.09,240.0,-1.00,1.07,30.51,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.75,5.48,13.03,19.23,20680.0,8252.0,158712.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.93,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-1.16,-28.02,56.86,12.90,89423.0,-61227.0,157269.0,131.97,60.0,H-SC,8.97,173.0,-0.68,1.12,30.77,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,2.56,-18.63,62.26,32.03,66789.0,-24561.0,107274.0,-12.01,58.0,H-SC,6.55,157.0,-0.37,0.77,20.16,OX40N,NTT,IT
40,INDUSINDBK,1800.00,0.72,-29.77,89.21,32.89,48373.0,-22982.0,54224.0,-769.87,74.0,L-MC,9.20,259.0,-0.48,0.39,49.33,XR,NTT,BANKS
17,COALINDIA,484.83,-0.75,5.48,13.03,19.23,20680.0,8252.0,158712.0,24.27,66.0,L-LC,9.22,182.0,0.40,1.13,27.93,XY25,ATH,MINING
57,RECLTD,446.00,0.53,0.66,19.94,20.74,40781.0,1348.0,204518.0,33.49,57.0,M-MC,5.18,189.0,0.03,1.46,11.35,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.49
1,25,44.56
2,50,76.46


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.76
MC    30.45
LC    26.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.27
X40      24.05
X40N     12.74
XY25     10.82
XR        8.99
AR        8.40
OX40N     7.75
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.10
X-LC    23.26
H-SC    22.87
M-SC    11.22
X-SC     7.87
H-MC     4.50
M-MC     1.46
M-LC     1.33
L-LC     1.13
H-LC     1.04
L-SC     0.80
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.17,-9.52,45.45
IT,12.93,-18.16,77.76
FINANCE,11.12,-14.59,62.68
MISC,7.10,-30.47,83.84
ELECTRICAL,5.80,-15.57,57.90
PAINTS,5.79,-13.90,30.81
INSURANCE,4.70,-2.95,38.18
PHARMA,3.91,-7.43,41.24
AUTO,2.80,-33.33,79.56


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3291905.0,21
AR,1395575.0,10
XR,1233357.0,12
X40,1153646.0,15
X40N,958258.0,9
OX40N,706230.0,10
XY25,425542.0,7
SR,286556.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3590385.0,24
X-MC,1660204.0,16
M-SC,1486821.0,15
X-LC,966257.0,13
X-SC,816128.0,8
H-MC,447909.0,3
L-SC,179432.0,2
M-LC,105988.0,1
H-LC,88111.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1262821.0      6
           AR         935769.0      5
M-SC       XY24       853462.0      6
H-SC       XR         800095.0      7
X-MC       X40        540057.0      7
           XY24       460166.0      3
           X40N       431330.0      4
X-LC       X40        418300.0      6
X-SC       X40N       320217.0      3
M-SC       OX40N      309745.0      5
H-SC       OX40N      305144.0      4
X-SC       XY24       300622.0      3
H-SC       SR         286556.0      2
H-MC       AR         238891.0      2
X-MC       XY25       228651.0      2
H-MC       XY24       209018.0      1
X-LC       X40N       206711.0      2
           XY24       205816.0      2
X-SC       X40        195289.0      2
M-SC       XR         190810.0      2
X-LC       XY25       135430.0      3
M-SC       AR         132804.0      2
M-LC       XR         105988.0      1
L-SC       OX40N       91341.0      1
H-LC       AR          88111.0      1
L-SC       XR          88091.0      1
L-MC       XR          48373.0      1
M-MC       XY25        40781.0      1
L-LC       XY25        20680.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
